In [4]:
%matplotlib
import gmaps
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import os
import pandas as pd
pd.options.display.max_columns = None
import requests
import time
from scipy.stats import linregress, pearsonr
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
#from sklearn.decomposition import FactorAnalysis

Using matplotlib backend: Qt5Agg


In [5]:
# Variable Units

# Per Capita Income, Birth Rate per (1k),
# General Fertility Rate per (1k),% Low Birth Weight,% Prenatal Care in First Trimester,
# % Preterm Births,Teen Birth Rate per (1k),Homicides per (100k),Breast Cancer Females per (100k),
# Cancer (All Sites) per (100k),Colorectal Cancer per (100k),Diabetes-related per (100k),Firearm-related per (100k),
# Infant Mortality Rate per (1k),Lung Cancer per (100k),Prostate Cancer in Males per (100k),
# Stroke (Cerebrovascular Disease) per (100k),Childhood Blood Lead Level Screening per (1k),
# Childhood Lead Poisoning per (100),Gonorrhea in Females per (100k),Gonorrhea in Males per (100k),
# Tuberculosis per (100k),% Below Poverty Level,% Crowded Housing,% Dependency,% No High School Diploma,Unemployment

In [6]:
# Final analysis data

final_df = pd.read_csv('output_data/analysis_data.csv')
# 77 rows × 20 columns

In [7]:
final_df.columns

Index(['Community Area Name', 'Homicide_rate_per_100k', 'Homicide', 'Violence',
       'Property Crimes', 'Breast cancer in females', 'Cancer (All Sites)',
       'Colorectal Cancer', 'Diabetes-related', 'Lung Cancer',
       'Prostate Cancer in Males', 'Stroke (Cerebrovascular Disease)',
       'Birth Rate', 'General Fertility Rate', 'Low Birth Weight',
       'Preterm Births', 'Teen Birth Rate', 'Below Poverty Level',
       'Dependency', 'Unemployment'],
      dtype='object')

In [8]:
health_factors = final_df[['Homicide_rate_per_100k', 'Breast cancer in females', 'Cancer (All Sites)',
       'Colorectal Cancer', 'Diabetes-related', 'Lung Cancer', 'Prostate Cancer in Males', 'Stroke (Cerebrovascular Disease)']]

health_factors.columns = ['Homicide (per 100k)', 'Breast Cancer', 'Cancer (All Sites)', 'Colorectal Cancer',\
                                 'Diabetes-related', 'Lung Cancer', 'Prostate Cancer', 'Stroke']

health_factors.corr().style.background_gradient()

,Homicide (per 100k),Breast Cancer,Cancer (All Sites),Colorectal Cancer,Diabetes-related,Lung Cancer,Prostate Cancer,Stroke
Homicide (per 100k),1.000000,0.342735,0.749447,0.650992,0.692539,0.706495,0.738349,0.780591
Breast Cancer,0.342735,1.000000,0.557138,0.455001,0.326569,0.328028,0.430277,0.303131
Cancer (All Sites),0.749447,0.557138,1.000000,0.804605,0.730662,0.900090,0.825934,0.692949
Colorectal Cancer,0.650992,0.455001,0.804605,1.000000,0.562055,0.722877,0.616068,0.592772
Diabetes-related,0.692539,0.326569,0.730662,0.562055,1.000000,0.633578,0.670423,0.672931
Lung Cancer,0.706495,0.328028,0.900090,0.722877,0.633578,1.000000,0.741645,0.719298
Prostate Cancer,0.738349,0.430277,0.825934,0.616068,0.670423,0.741645,1.000000,0.641115
Stroke,0.780591,0.303131,0.692949,0.592772,0.672931,0.719298,0.641115,1.000000


In [9]:
birth_factors = final_df[['Homicide_rate_per_100k',
       'Birth Rate', 'General Fertility Rate', 'Low Birth Weight',
       'Preterm Births', 'Teen Birth Rate']].reset_index(drop=True)


birth_factors.rename(columns={'Homicide_rate_per_100k': 'Homicide (per 100k)'}, inplace=True)

birth_factors.corr()

,Homicide (per 100k),Birth Rate,General Fertility Rate,Low Birth Weight,Preterm Births,Teen Birth Rate
Homicide (per 100k),1.000000,0.188536,0.293843,0.747493,0.742019,0.768307
Birth Rate,0.188536,1.000000,0.810334,0.108179,0.004334,0.612710
General Fertility Rate,0.293843,0.810334,1.000000,0.142189,0.122235,0.655528
Low Birth Weight,0.747493,0.108179,0.142189,1.000000,0.843100,0.622936
Preterm Births,0.742019,0.004334,0.122235,0.843100,1.000000,0.549843
Teen Birth Rate,0.768307,0.612710,0.655528,0.622936,0.549843,1.000000


In [10]:
economic_factors = final_df[['Homicide_rate_per_100k','Below Poverty Level','Dependency', 'Unemployment']].reset_index(drop=True)
economic_factors.rename(columns={'Homicide_rate_per_100k': 'Homicide (per 100k)'}, inplace=True)
economic_factors.corr().corr().style.background_gradient('BrBG')

,Homicide (per 100k),Below Poverty Level,Dependency,Unemployment
Homicide (per 100k),1.000000,0.166107,-0.441106,0.596215
Below Poverty Level,0.166107,1.000000,-0.954319,0.494276
Dependency,-0.441106,-0.954319,1.000000,-0.562269
Unemployment,0.596215,0.494276,-0.562269,1.000000


In [56]:
# Function for Simple Regression

# Features: plots line, provides stats, describes relationship

def regression(x, y, x_label, y_label):
    title = f'{x_label} predicting {y_label}'
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x, y)
    regress_values = x * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    plt.scatter(x, y)
    plt.plot(x,regress_values,"r-")
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(title)
    title = title.replace('\n', '_').replace(' ', '_')
    plt.savefig(f'output_data/{title}.png')
    plt.show()
    #plt.close()
    print(line_eq)
    print()
    r = pearsonr(x, y)[0]
    p = pearsonr(x, y)[1]
    print(f"r = {r:.2f}, p = {p:.4f}, r_squared = {rvalue**2:.2f}")
    print()
    if p > 0.05:
        print(f'There is no relationship between {x_label.lower()} and {y_label.lower()}, p > 0.05.')
    else:
        if r > 0:
            explanation = f'An increase in {x_label.lower()} is associated with an increase in {y_label.lower()}.'
            direction = 'positive'
            if r >= 0.6:
                strength = 'strong'
            elif r >= 0.3:
                strength = 'moderate'
            else:
                strength = 'weak'
        else:
            direction = 'negative'
            explanation = f'An increase in {x_label.lower()} is associated with a decrease in {y_label.lower()}.'
            if r <= -0.6:
                strength = 'strong'
            elif r <= -0.3:
                strength = 'moderate'
            else:
                strength = 'weak'       
        print(f'There is a {strength} {direction} relationship between {x_label.lower()} and {y_label.lower()}, p > 0.05.')
        print(explanation)
    print()

In [12]:
# Multiple Regression Function

# Features: Provides stats and statistical model test

def multiple_regression(xs, y): # xs is a dataframe, y is a series; prints output and returns predictions
    xs = sm.add_constant(xs) # adding a constant
    model = sm.OLS(y, xs).fit()
    predictions = model.predict(xs) 
    print_model = model.summary()
    print(print_model)
    return predictions

In [47]:
# 3D Visualization of Multiple Regression Function
def regression_3d_visualization(dataframe, x1, x2, y, size = (10,10), x1label='X Label', x2label='Y Label', ylabel='Z Label'):
    df = dataframe[[x1, x2, y]].reset_index(drop=True)
    x1r, x2r, yr = x1, x2, y
    if ' ' in x1:
        x1r = x1.replace(' ', '')
        df.rename(columns={x1: x1r}, inplace=True)
    if ' ' in x2:
        x2r = x2.replace(' ', '')
        df.rename(columns={x2: x2r}, inplace=True)
    if ' ' in y:
        yr = y.replace(' ', '')
        df.rename(columns={y: yr}, inplace=True)
    model = smf.ols(formula=f'{yr} ~ {x1r} + {x2r}', data=df)
    results = model.fit()
    results.params

    x_dim, y_dim = np.meshgrid(np.linspace(df[x1r].min(), df[x1r].max(), 100), np.linspace(df[x2r].min(), df[x2r].max(), 100))
    xs = pd.DataFrame({x1r: x_dim.ravel(), x2r: y_dim.ravel()})
    predicted_y = results.predict(exog=xs)
    predicted_y=np.array(predicted_y)

    fig = plt.figure(figsize=size, facecolor='b')
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(df[x1r], df[x2r], df[yr], c='red', marker='o', alpha=0.5)
    ax.plot_surface(x_dim, y_dim, predicted_y.reshape(x_dim.shape), color='b', alpha=0.3)
    ax.set_xlabel(x1label)
    ax.set_ylabel(x2label)
    ax.set_zlabel(ylabel)
    plt.show()

In [60]:
#Regressing neighborhood homicide rate (per 1k) on the teen birth rate (per 1k).

#regression(x, y, x_label, y_label)

regression(final_df['Teen Birth Rate'], final_df['Homicide_rate_per_100k'], 'Teen Birth Rate (per 1k)', 'Neighborhood Homicide Rate (per 100k)')

y = 0.0x + -0.05

r = 0.77, p = 0.0000, r_squared = 0.59

There is a strong positive relationship between teen birth rate (per 1k) and neighborhood homicide rate (per 1k), p > 0.05.
An increase in teen birth rate (per 1k) is associated with an increase in neighborhood homicide rate (per 1k).



In [15]:
#Regressing neighborhood homicide rate (per 100k) on Unemployment (%)

#regression(x, y, x_label, y_label)

regression(final_df['Unemployment'], final_df['Homicide_rate_per_100k'], 'Unemployment (%)', 'Neighborhood Homicide Rate (per 100k)')


y = 1.92x + -7.46

r = 0.81, p = 0.0000, r_squared = 0.66

There is a strong positive relationship between unemployment (%) and neighborhood homicide rate (per 100k), p > 0.05.
An increase in unemployment (%) is associated with an increase in neighborhood homicide rate (per 100k).



In [16]:
#Multiple Regression: Regressing Homicide_rate_per_100k on Teen Birth & Economic Factors
xs = final_df[['Teen Birth Rate', 'Unemployment']]
y = final_df['Homicide_rate_per_100k']

multiple_regression(xs, y)

#Note the statistically significant Kurtosis

                              OLS Regression Results                              
Dep. Variable:     Homicide_rate_per_100k   R-squared:                       0.735
Model:                                OLS   Adj. R-squared:                  0.728
Method:                     Least Squares   F-statistic:                     102.8
Date:                    Tue, 28 Jul 2020   Prob (F-statistic):           4.37e-22
Time:                            15:21:45   Log-Likelihood:                -273.72
No. Observations:                      77   AIC:                             553.4
Df Residuals:                          74   BIC:                             560.5
Df Model:                               2                                         
Covariance Type:                nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
co

0     11.313539
1     19.267911
2      8.283757
3      9.665290
4     39.426124
        ...    
72     9.800967
73    26.779837
74     6.627526
75     8.531502
76    23.413070
Length: 77, dtype: float64

In [17]:
#teen birth rate
#below poverty level
#unemployment

# 3D Visualization: Regressing Neighborhood Homicide Rate (per 100k)on Teen Birth Rate and Unemployment
regression_3d_visualization(dataframe=final_df, x1='Unemployment', x2='Teen Birth Rate', y='Homicide_rate_per_100k',\
                            size = (10,10), x1label='Unemployment', x2label='Teen Birth Rate (per 1k)',\
                            ylabel='Neighborhood Homicide_rate_per_100k')

In [18]:
#Example prediction equation
t, e = 50.064935 + (2*28.097817), 13.303896 + (2*7.031965)
Neighborhood_Homicide_Rate_Per_100k = -10.1931 + 0.2246*t + 1.2792*e
print(Neighborhood_Homicide_Rate_Per_100k)
#18.068831 + (2*16.561077) = 51.190985

48.6819468166


In [19]:
#Predicted and Actual Homicides based on Teen Birth Rate and Unemployment

predicted_homicide_rate = []

for index, row in final_df.iterrows():
    predicted_value = -10.1931 + 0.2246*row['Teen Birth Rate'] + 1.2792*row['Unemployment']
    predicted_homicide_rate.append(predicted_value)
    

actual_homicide_rate = final_df['Homicide_rate_per_100k'].to_list()

In [20]:
#predicted_homicide_rate
#actual_homicide_rate 
N = len(predicted_homicide_rate) #77
ind = np.arange(N) 
width = .35
f, ax = plt.subplots(figsize=(8.5,3.5))
plt.bar(ind, predicted_homicide_rate, width, label='Predicted (per 100k)')
plt.bar(ind + width, actual_homicide_rate, width, label='Observed (per 100k)')

plt.ylabel('Homidicide Rates (per 100k)')
plt.xlabel('Neighborhoods (n = 77)')
plt.title('Predicted vs. Observed Homicide Rate for each Chicago Neighborhood')
neighborhoods = ()
for i in range(77):
    neighborhoods += ("",)

plt.xticks(ind + width / 2, neighborhoods)
plt.legend(loc='best')
plt.show()

In [21]:
"""
Limitations:

Correlational and a cross_sectional non-experimental study
Drawing inferences about individuals based on aggregated data (e.g., aggregated at neighborhood and state level)
Well-being measure: 
  need more measures of health factors
  need psychological measures
  Need more measures economic measures

Non-normality of outcome variable

Future Directions:

Hierarchical Regression Analysis 
Exploratory Factor Analysis
Individual-level data

"""

'\nLimitations:\n\nCorrelational and a cross_sectional non-experimental study\nDrawing inferences about individuals based on aggregated data (e.g., aggregated at neighborhood and state level)\nWell-being measure: \n  need more measures of health factors\n  need psychological measures\n  Need more measures economic measures\n\nNon-normality of outcome variable\n\nFuture Directions:\n\nHierarchical Regression Analysis \nExploratory Factor Analysis\nIndividual-level data\n\n'

In [66]:
final_df['Homicide_rate_per_100k'].hist(bins=20)

In [44]:
#log transformation + constant to make homicide variable more normally distributed
final_df['Homicide_rate_log'] = final_df['Homicide_rate_per_100k'].apply(lambda x: np.log(x+1))

In [67]:
final_df['Homicide_rate_log'].hist(bins=20)

In [64]:
#log units

#Regressing neighborhood homicide rate (per 100k) on the teen birth rate (per 1k).

#regression(x, y, x_label, y_label)

regression(final_df['Teen Birth Rate'], final_df['Homicide_rate_log'], 'Teen Birth Rate (per 1k)', 'Log Units - Homicide Rate')

y = 0.03x + 1.05

r = 0.83, p = 0.0000, r_squared = 0.68

There is a strong positive relationship between teen birth rate (per 1k) and log units - homicide rate, p > 0.05.
An increase in teen birth rate (per 1k) is associated with an increase in log units - homicide rate.



In [71]:
#log units

#Regressing neighborhood homicide rate (per 100k) on the Unemployment Rate.

#regression(x, y, x_label, y_label)

regression(final_df['Unemployment'], final_df['Homicide_rate_log'], 'Unemployment (%)', 'Log Units - Homicide Rate')

y = 0.11x + 1.02

r = 0.79, p = 0.0000, r_squared = 0.63

There is a strong positive relationship between unemployment (%) and log units - homicide rate, p > 0.05.
An increase in unemployment (%) is associated with an increase in log units - homicide rate.



In [72]:
#Log Units

#Multiple Regression: Regressing Homicide_rate_per_100k on Teen Birth & Economic Factors
xs = final_df[['Teen Birth Rate', 'Unemployment']]
y = final_df['Homicide_rate_log']

multiple_regression(xs, y)

#No more statistically significant kurtosis.

                            OLS Regression Results                            
Dep. Variable:      Homicide_rate_log   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.762
Method:                 Least Squares   F-statistic:                     122.7
Date:                Tue, 28 Jul 2020   Prob (F-statistic):           3.14e-24
Time:                        15:56:33   Log-Likelihood:                -52.148
No. Observations:                  77   AIC:                             110.3
Df Residuals:                      74   BIC:                             117.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.7939      0.124     

0     2.160981
1     2.575383
2     1.779882
3     2.032343
4     3.780465
        ...   
72    2.092473
73    3.069611
74    1.821159
75    2.073619
76    2.772318
Length: 77, dtype: float64

In [73]:
#Log units

#teen birth rate
#below poverty level
#unemployment

# 3D Visualization: Regressing Neighborhood Homicide Rate (per 100k)on Teen Birth Rate and Unemployment
regression_3d_visualization(dataframe=final_df, x1='Unemployment', x2='Teen Birth Rate', y='Homicide_rate_log',\
                            size = (10,10), x1label='Unemployment', x2label='Teen Birth Rate (per 1k)',\
                            ylabel='Neighborhood Homicide Rate in Log Units')